In [3]:
import re

In [4]:
# Extract student data with booklet id
IDBOOK = 17    
data_ID = []
data = []
with open('data.csv', 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        d = re.split(",|\n", line)
        if i == 0:
            data_ID = d
        elif d[1] == str(IDBOOK):
            if len(d) != len(data_ID):
                print(len(d), len(data_ID))
                raise Exception(f'Parsing Error at line {i+1}')  
            stud = {}
            for j, id in enumerate(data_ID):
                if d[j] != ' ':
                    stud[id] = d[j]
            data.append(stud)

print(data[0])
print(data[-1])

{'\ufeffIDCNTRY': '410', 'IDBOOK': '17', 'IDSCHOOL': '5003', 'IDCLASS': '500302', 'IDSTUD': '50030203', 'MP61026': '3', 'MP61273': '4', 'MP61034': '10', 'MP61040': '1', 'MP61228': '11', 'MP61166': '10', 'MP61171': '1', 'MP61080': '10', 'MP61222': '4', 'MP61076': '10', 'MP61084': '10', 'MP61275': '2', 'MP61027': '79', 'MP61255': '79', 'MP61021': '10', 'MP61043': '10', 'MP61151': '4', 'MP61172': '1', 'MP61223': '3', 'MP61269': '4', 'MP61081A': '10', 'MP61081B': '10', '': ''}
{'\ufeffIDCNTRY': '410', 'IDBOOK': '17', 'IDSCHOOL': '5169', 'IDCLASS': '516905', 'IDSTUD': '51690522', 'MP61026': '3', 'MP61273': '4', 'MP61034': '79', 'MP61040': '1', 'MP61228': '79', 'MP61166': '10', 'MP61171': '1', 'MP61080': '10', 'MP61222': '4', 'MP61076': '10', 'MP61084': '10', 'MP61275': '2', 'MP61027': '10', 'MP61255': '20', 'MP61021': '10', 'MP61043': '10', 'MP61151': '4', 'MP61172': '1', 'MP61223': '3', 'MP61269': '4', 'MP61081A': '10', 'MP61081B': '79', '': ''}


In [5]:
# Transform data to [1=Correct, -1=Wrong, 0=Invalid]
def transform(stud_id, prob_id, prob_type, response, answer=''):
    res = int(response)
    ret = -10
    if prob_type == 'CR':
        if res == 10 or res == 11 or res == 20:
            ret = 1
        elif res == 70 or res == 79:
            ret = -1
        elif res == 96 or res == 99:
            ret = 0
        else:
            raise Exception(f'Invalid response of student {stud_id} for {prob_type} problem {prob_id}.')
        
    elif prob_type == 'MC':
        if res == 6 or res == 9:
            ret = 0
        elif ord(answer) - 64 == res:
            ret = 1
        elif res in [1, 2, 3, 4]:
            ret = -1
        else:
            raise Exception(f'Invalid response of student {stud_id} for {prob_type} problem {prob_id}.')
    return ret

In [6]:
# Get answer of MC problems & type of problems
content = 'Number'
MC_answer = {}
prob_type = {}
with open('Item_Information.csv', 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        #print(line)
        d = re.split(",|\n", line)
        if i == 0 or d[3] != content: continue
        prob_id = d[0]
        prob_type[prob_id] = d[1]
        if d[1] == 'MC':
            answer = d[2]
            MC_answer[prob_id] = answer
print(f'    Answer for MC Problems: {MC_answer}')
print(f'    Type of Problems: {prob_type}')

    Answer for MC Problems: {'MP51040': 'A', 'MP61026': 'C', 'MP61273': 'D', 'MP61040': 'A', 'MP61171': 'A', 'MP61222': 'D', 'MP51052': 'C', 'MP51049': 'B', 'MP51098': 'D', 'MP61018A': 'A', 'MP61018B': 'A', 'MP61018C': 'B', 'MP61018D': 'A', 'MP61274': 'B', 'MP61179': 'C', 'MP61052': 'D', 'MP51075': 'C', 'MP51226': 'B', 'MP51103': 'C', 'MP61275': 'B', 'MP61151': 'D', 'MP61172': 'A', 'MP61223': 'C', 'MP61246': 'C', 'MP61049': 'C', 'MP61232': 'A', 'MP61240A': 'A', 'MP61240B': 'A', 'MP61240C': 'B', 'MP61244': 'C', 'MP61041': 'B', 'MP61252': 'D'}
    Type of Problems: {'MP51043': 'CR', 'MP51040': 'MC', 'MP51008': 'CR', 'MP51031A': 'CR', 'MP51031B': 'CR', 'MP51508': 'CR', 'MP61026': 'MC', 'MP61273': 'MC', 'MP61034': 'CR', 'MP61040': 'MC', 'MP61228': 'CR', 'MP61166': 'CR', 'MP61171': 'MC', 'MP61222': 'MC', 'MP51206': 'CR', 'MP51052': 'MC', 'MP51049': 'MC', 'MP51098': 'MC', 'MP51030': 'CR', 'MP61018': 'CR', 'MP61018A': 'MC', 'MP61018B': 'MC', 'MP61018C': 'MC', 'MP61018D': 'MC', 'MP61274': 'MC'

In [8]:
valid_problem_ID = []
for id in data_ID:
  if id[:2] == 'MP' and id in data[0].keys() and id in prob_type.keys():
    valid_problem_ID.append(id)
print(valid_problem_ID)

# Save data
with open(f'{IDBOOK}_{content}_data.csv', 'w') as f:
  f.write('IDSTUD')
  for id in valid_problem_ID:
    f.write(f', {id}')
  f.write('\n')
  for d in data:
    print(d)
    stud_id = d['IDSTUD']
    f.write(stud_id)
    for id in valid_problem_ID:
      if prob_type[id] == 'MC':
        f.write(f', {transform(stud_id, id, prob_type[id], d[id], MC_answer[id])}')
      else:
        f.write(f', {transform(stud_id, id, prob_type[id], d[id])}')
    f.write('\n')

204', 'IDSTUD': '51420413', 'MP61026': '3', 'MP61273': '4', 'MP61034': '10', 'MP61040': '1', 'MP61228': '79', 'MP61166': '10', 'MP61171': '1', 'MP61080': '79', 'MP61222': '4', 'MP61076': '10', 'MP61084': '10', 'MP61275': '2', 'MP61027': '10', 'MP61255': '20', 'MP61021': '10', 'MP61043': '10', 'MP61151': '4', 'MP61172': '1', 'MP61223': '3', 'MP61269': '4', 'MP61081A': '10', 'MP61081B': '79', '': ''}
{'\ufeffIDCNTRY': '410', 'IDBOOK': '17', 'IDSCHOOL': '5142', 'IDCLASS': '514204', 'IDSTUD': '51420421', 'MP61026': '3', 'MP61273': '4', 'MP61034': '79', 'MP61040': '3', 'MP61228': '20', 'MP61166': '10', 'MP61171': '1', 'MP61080': '10', 'MP61222': '4', 'MP61076': '79', 'MP61084': '79', 'MP61275': '2', 'MP61027': '10', 'MP61255': '79', 'MP61021': '79', 'MP61043': '79', 'MP61151': '4', 'MP61172': '2', 'MP61223': '3', 'MP61269': '4', 'MP61081A': '79', 'MP61081B': '99', '': ''}
{'\ufeffIDCNTRY': '410', 'IDBOOK': '17', 'IDSCHOOL': '5143', 'IDCLASS': '514302', 'IDSTUD': '51430201', 'MP61026': '3', 